# Adquisición de datos para finanzas

## 1. Adquisición de datos a partir de ficheros

### 1.1 Ficheros separados por coma (CSV).

In [ ]:
# Importación de paquetes
import pandas as pd

Utilizaremos diferentes métodos de Pandas para leer los distintos tipos de ficheros. Para CSV se emplea ```read_csv()```. Si no se especifica usa el separador por defecto **","**.

https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

In [ ]:
invoices_df = pd.read_csv('../data/ecommerce.csv')
invoices_df.head()

Cuando el separador no es **","** hay que definirlo explícitamente. Puede ser **';'**, tabulación, **'/'** u otros.

En caso contrario no leerá correctamente los datos.

In [ ]:
invoices_semicolon_sep_df = pd.read_csv('../data/ecommerce_semicolon_sep.csv')
invoices_semicolon_sep_df.head()

La forma de especificar el separador es con el parámetro ```sep```.

In [ ]:
invoices_semicolon_sep_df = pd.read_csv('../data/ecommerce_semicolon_sep.csv', sep=';')
invoices_semicolon_sep_df.head()

### 1.2. Ficheros de Excel.


En el caso de los ficheros xlsx se utiliza el método ```read_excel()```. Es necesario especificar la hoja de la que vamos a extraer los datos.

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [ ]:
invoices_df = pd.read_excel('../data/ecommerce_excel.xlsx', sheet_name="2024")
invoices_df.head()

### 1.3. Ficheros JSON.

Para los ficheros JSON utilizaremos ```read_json()```. Más adelante, en el apartado de API veremos algunas particularidades de la lectura de esta forma de estructurar los datos.

https://pandas.pydata.org/docs/reference/api/pandas.read_json.html

In [ ]:
ecommerce_json_df = pd.read_json('../data/ecommerce.json')
ecommerce_json_df.head()

### 1.4. Ficheros en formato parquet.

Los ficheros de [Apache Parquet](https://parquet.apache.org/) tienen el inconveniente de no ser legibles por un ser humano, debido a la forma en que estructura los datos y metadatos.

Es necesario instalar y especificar el motor de serialización. Los más frecuentes son [Apache Arrow](https://arrow.apache.org/) y [Fast Parquet](https://fastparquet.readthedocs.io/en/latest/). Lo concretamos con el parámetro ```engine```.

https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html

In [ ]:
ecommerce_parquet_df = pd.read_parquet('../data/ecommerce.parquet', engine='pyarrow')
ecommerce_parquet_df.head()

## 2. Adquisición de datos a través de APIs.

In [ ]:
# Importación de paquetes
import requests
from pandas import json_normalize

Necesitamos importar la librería **requests**, que nos permitirá realizar peticiones a la API.

En este caso el método que necesitaremos para extraer los datos es ```get()```. En las peticiones necesitamos especificar varios parámetros:
* **url**: dirección web a la que apunta la petición.
* **headers**: configuración de cabeceras para la petición.
* **querystring**: parámetros que se añaden a la url.

Si utilizáramos una petición de tipo ```post()``` necesitaríamos concretar un **body**.

In [ ]:
url = "https://real-time-product-search.p.rapidapi.com/search"

querystring = {"q":"Nike shoes","country":"us","language":"en","limit":"30"}

headers = {
	"X-RapidAPI-Key": "be814bcabbmshc4f57ebcf4b7568p1eb15djsn52335224755f",
	"X-RapidAPI-Host": "real-time-product-search.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring).json()
print(response)

De toda la respuesta solo necesitamos los datos, que se encuentran en el atributo **data** en este caso. Por lo tanto lo extraemos para procesarlo, teniendo en cuenta que es de tipo diccionario.

In [ ]:
data_dict = response["data"]
print(data_dict)

Para procesar los datos como Dataframe de Pandas se utilizará el constructor de la clase **Dataframe**. En este caso se pasa el diccionario como parámetro, y seleccionamos los atributos que queramos procesar.

In [ ]:
selected_cols = [
    'product_id',
    'product_title',
    'product_rating',
    'typical_price_range',
    'offer'
]
data_df = pd.DataFrame(data_dict)[selected_cols]
data_df.head()

Como vemos hay un problema en la columna **offer**. Al estar anidado un objeto en este atributo necesitamos aplanarla, generando columnas por cada uno de esos atributos. Para aplanarlo utilizamos ```json_normalize()``` con nuestra columna.

https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html

Una vez aplanado, unimos las nuevas columnas con el dataframe original para tener el registro completo sin anidamientos. Para unir ambos dataframes utilizaremos ```concat()```.

https://pandas.pydata.org/docs/reference/api/pandas.concat.html

In [ ]:
# Aplanar el diccionario dentro de la columna 'datos'
df_aplanado = json_normalize(data_df['offer'])

# Concatenar el DataFrame aplanado con el DataFrame original
df_resultante = pd.concat([data_df, df_aplanado], axis=1)

cols_to_drop = [
    'offer',
    'offer_page_url',
    'store_reviews_page_url',
    'original_price',
    'product_condition',
    'buy_now_url',
    'on_sale',
    'shipping'
]

# Eliminamos columnas innecesarias
df_resultante = df_resultante.drop(columns=cols_to_drop, axis=1)

df_resultante.head()

## 3. Adquisición de datos procedentes de bases de datos relacionales.

**Nota**: Se ha generado una BBDD PostgreSQL en https://console.neon.tech/app/projects de forma gratuita para este caso. Se han insertado 18 registros del CSV de ecommerce trabajado previamente para realizar pruebas.

Para poder conectarnos con BBDD relacionales utilizaremos los paquetes ```postgresql``` y ```sqlalchemy```. Con ellos generaremos una conexión con nuestra BBDD, utilizando la configuración del objeto ***URL**.

Con ```create_engine()``` generamos una conexión con nuestra BBDD relacional, de modo que sea posible realizar cosultas SQL.

In [ ]:
from sqlalchemy import create_engine, URL

url_object = URL.create(
    "postgresql",
    username="ismaelcazalilla",
    password="l10EaBKMzjJU",
    host="ep-throbbing-haze-36918596.eu-central-1.aws.neon.tech",
    database="adquisicion_datos",
)

# Generamos una instancia de motor de conexión a la base de datos
db_engine = create_engine(url_object)

Una vez establecida la conexión se utiliza el método ```read_sql()```, en el que escribiremos la sentencia SQL, así como el motor de BBDD.

https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html#pandas.read_sql

In [ ]:
# Conectamos con la base de datos y lanzamos una query para leer los datos
with db_engine.connect() as conn, conn.begin():  
    df = pd.read_sql("SELECT * FROM adquisicion.ecommerce WHERE invoiceno = '536365'", con=db_engine)
    
df.head()